In [62]:
#numpy loader method to load numpy arrays into the Pytorch ImageFolder dataset
import numpy as np

def my_numpy_loader(filename):
    return np.load(filename)

In [63]:
import visdom
import numpy as np
viz = visdom.Visdom()

In [69]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import visdom


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#Visdom Configuration
viz = visdom.Visdom()

# Hyper parameters
num_epochs = 15
num_classes = 2
batch_size = 100
learning_rate = 0.001

# Dicom dataset
train_dataset = torchvision.datasets.DatasetFolder(root='2D/train', 
                                           transform=transforms.ToTensor(), loader=my_numpy_loader, extensions="npy")

test_dataset = torchvision.datasets.DatasetFolder(root='2D/test', 
                                          transform=transforms.ToTensor(), loader=my_numpy_loader, extensions="npy")
# print(train_dataset)
# print(test_dataset)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvNet,self).__init__()
        #Convo 1
        self.cnn1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.batch_norm1 = nn.BatchNorm2d(16)
        self.relu1=nn.ReLU()
        #Maxpool_1
        self.maxpool1=nn.MaxPool2d(kernel_size=2)
        #Convo_2
        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.batch_norm2 = nn.BatchNorm2d(32)
        self.relu2=nn.ReLU()
        #Maxpool_2
        self.maxpool2=nn.MaxPool2d(kernel_size=2)

        #Linear Layer
        self.fc1=nn.Linear(8192,1024)
        self.fc2=nn.Linear(1024,32)
        self.fc3=nn.Linear(32,2)

    def forward(self,x):
        #Convo_1
        out=self.cnn1(x)
        #print(out.shape)
        out=self.batch_norm1(out)
        #print(out.shape)
        out=self.relu1(out)
        #print(out.shape)
        
        #Max_pool1
        out=self.maxpool1(out)
        #print(out.shape)
        
        #Convo_2
        out=self.cnn2(out)
        #print(out.shape)
        out=self.batch_norm2(out)
        #print(out.shape)
        out=self.relu2(out)
        #print(out.shape)
        
        #Max_pool2
        out=self.maxpool2(out)
        #print(out.shape)
        out=out.view(out.size(0),-1)#Flattening out 
        #print(out.shape)
        
        #Linear Layers
        out=self.fc1(out)
        #print(out.shape)
        out=self.fc2(out)
        #print(out.shape)
        out=self.fc3(out)
        #print(out.shape)
        return out

model = ConvNet(num_classes).to(device).double()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
loss_window = vis.line(
    Y=torch.zeros((1)).cpu(),
    X=torch.zeros((1)).cpu(),
    opts=dict(xlabel='Epoch',ylabel='Loss',title='Training Loss'))

accuracy_window = vis.line(
    Y=torch.zeros((1)).cpu(),
    X=torch.zeros((1)).cpu(),
    opts=dict(xlabel='Epoch',ylabel='Accuracy [%]',title='Network Accuracy'))


counter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        loss_array = np.array([loss.item()])
        viz.line(X=torch.ones((1,1)).cpu()*counter, Y=torch.Tensor([loss]).unsqueeze(0),win=loss_window,update='append')
        counter = counter + 1

    # Test the model
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test Accuracy of the model on the 212 test images: {} %'.format(100 * correct / total))
        acc = 100 * correct / total
        viz.line(X=torch.ones((1,1)).cpu()*counter, Y=torch.Tensor([acc]).unsqueeze(0),win=accuracy_window,update='append')

# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')

Dataset DatasetFolder
    Number of datapoints: 982
    Root Location: 2D/train
    Transforms (if any): ToTensor()
    Target Transforms (if any): None
Dataset DatasetFolder
    Number of datapoints: 422
    Root Location: 2D/test
    Transforms (if any): ToTensor()
    Target Transforms (if any): None
Epoch [1/15], Step [1/10], Loss: 0.7883
Epoch [1/15], Step [2/10], Loss: 6.3802
Epoch [1/15], Step [3/10], Loss: 4.0139
Epoch [1/15], Step [4/10], Loss: 0.6660
Epoch [1/15], Step [5/10], Loss: 2.1404
Epoch [1/15], Step [6/10], Loss: 0.6512
Epoch [1/15], Step [7/10], Loss: 1.3081
Epoch [1/15], Step [8/10], Loss: 1.5958
Epoch [1/15], Step [9/10], Loss: 1.4178
Epoch [1/15], Step [10/10], Loss: 0.6415
Test Accuracy of the model on the 212 test images: 52.60663507109005 %
Epoch [2/15], Step [1/10], Loss: 0.6858
Epoch [2/15], Step [2/10], Loss: 0.6992
Epoch [2/15], Step [3/10], Loss: 0.6975
Epoch [2/15], Step [4/10], Loss: 0.7209
Epoch [2/15], Step [5/10], Loss: 0.7145
Epoch [2/15], Step [6/1

In [15]:
from visdom import Visdom

import numpy as np

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class VisdomLinePlotter(object):
    """Plots to Visdom"""
    def __init__(self, env_name='main'):
        self.viz = Visdom()
        self.env = env_name
        self.plots = {}
    def plot(self, var_name, split_name, title_name, x, y):
        if var_name not in self.plots:
            self.plots[var_name] = self.viz.line(X=np.array([x,x]), Y=np.array([y,y]), env=self.env, opts=dict(
                legend=[split_name],
                title=title_name,
                xlabel='Epochs',
                ylabel=var_name
            ))
        else:
            self.viz.line(X=np.array([x]), Y=np.array([y]), env=self.env, win=self.plots[var_name], name=split_name, update = 'append')